In [ ]:
# https://blog.dataiku.com/how-to-perform-basic-ml-serving-with-python-docker-kubernetes
! curl -X POST localhost:8080/predict -d '{"data": [5.1, 3.5, 1.4, 0.2]}' -H 'Content-Type: application/json'
{
"prediction": "0"
}

# 1.From ChatGPT: How to build Flask API for predictions

In [ ]:
import os
from flask import Flask, request, jsonify
import pandas as pd
from sklearn.externals import joblib

app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    # get data from request
    data = request.get_json(force=True)
    # convert data to dataframe
    df = pd.DataFrame.from_dict(data)
    # load model
    model = joblib.load('./models/titanic_model.pkl')
    # make prediction
    prediction = model.predict(df)
    # return prediction
    return jsonify({'prediction': prediction.tolist()})

if __name__ == '__main__':
    app.run(port=5000, debug=True)

To use this script, you will need to have a trained machine learning model saved as a pickle file in the ./models folder. You can then start the Flask API by running python api.py.

You can test the API by making a POST request to the /predict endpoint with a JSON payload containing the features for which you want to make a prediction. For example, using curl, you can do:

In [ ]:
!curl -X POST -H "Content-Type: application/json" -d '{
    "Pclass": 3,
    "Sex": "male",
    "Age": 22,
    "SibSp": 1,
    "Parch": 0,
    "Fare": 7.25,
    "Embarked": "S"
    }' http://localhost:5000/predict

This should return a JSON response with the prediction for the input data.
e.g. 

In [ ]:
{
"prediction": "1"
}

# 2.For batch inference

In [ ]:
@app.route('/predict', methods=['POST'])
def predict():
    # get data from request
    data = request.get_json(force=True)
    # convert data to list of dictionaries
    samples = [sample for sample in data]
    # convert to dataframe
    df = pd.DataFrame.from_dict(samples)
    # load model
    model = joblib.load('./models/titanic_model.pkl')
    # make prediction
    prediction = model.predict(df)
    # return prediction
    return jsonify({'prediction': prediction.tolist()})

You can then make a batch prediction by sending a list of samples in the request payload, like this:

In [ ]:
!curl -X POST -H "Content-Type: application/json" -d '[
  {
    "Pclass": 3,
    "Sex": "male",
    "Age": 22,
    "SibSp": 1,
    "Parch": 0,
    "Fare": 7.25,
    "Embarked": "S"
  },
  {
    "Pclass": 3,
    "Sex": "female",
    "Age": 32,
    "SibSp": 0,
    "Parch": 0,
    "Fare": 7.9250,
    "Embarked": "S"
  }
]' http://localhost:5000/predict

This should return a list of predictions for each sample in the input list.

If you make a batch prediction with the modified predict function that I provided, the response will be a JSON object with a single key, prediction, that maps to a list of predictions, where each prediction is a string representing the predicted class (either "0" or "1" in this case, since it's a binary classification problem).

Here is an example of the response you might receive for the batch prediction request that I provided in the previous message:

`{
  "prediction": ["0", "1"]
}`


Each element in the prediction list corresponds to the prediction for one of the samples in the input list. In this example, the first sample is predicted to belong to class "0", and the second sample is predicted to belong to class "1".

In [ ]:
{
  "prediction": ["0", "1"]
}

# 3.To deploy the Flask API in a Docker container, 

you will need to do the following:

1. Write a Dockerfile that specifies the steps to build the Docker image. The Dockerfile should start with a base image, such as python:3, and then use pip to install the required Python libraries. It should also copy the API script and any other necessary files to the image.


Here is an example Dockerfile that shows how to build the API image:

In [ ]:
# Dockerfile
FROM python:3

# Install required libraries
RUN pip install pandas scikit-learn flask

# Copy API script
COPY api.py .

# Copy model file
COPY models/titanic_model.pkl models/

# Set the entrypoint to the API script
ENTRYPOINT ["python", "api.py"]

2. Build the Docker image using the Dockerfile. You can do this by running the following command in the same directory as the Dockerfile:

In [ ]:
!docker build -t titanic-api .

This will create a Docker image with the name titanic-api that contains the API and all the necessary dependencies.

3. Run the Docker image as a container using the following command:

In [ ]:
!docker run -p 5000:5000 titanic-api

This will start a container based on the titanic-api image and expose the API on port 5000 of the container. The -p 5000:5000 flag maps port 5000 of the container to port 5000 on the host machine, so you can access the API at http://localhost:5000.

That's it! You should now be able to access the API using the same requests as before, but now it will be running inside a Docker container.

Q: So after that we shall be connected to the bash interpreter of the docker container, and then we can still run exactly the same command to perform inference ? 

In [ ]:
!curl -X POST -H "Content-Type: application/json" -d '[
  {
    "Pclass": 3,
    "Sex": "male",
    "Age": 22,
    "SibSp": 1,
    "Parch": 0,
    "Fare": 7.25,
    "Embarked": "S"
  },
  {
    "Pclass": 3,
    "Sex": "female",
    "Age": 32,
    "SibSp": 0,
    "Parch": 0,
    "Fare": 7.9250,
    "Embarked": "S"
  }
]' http://localhost:5000/predict

A: No, you don't need to be connected to the bash interpreter of the Docker container to make predictions with the API. Once you have started the container by running docker run -p 5000:5000 titanic-api, the API will be running in the background and you can make requests to it from your host machine.

To make a prediction, you can use the same curl command that I provided earlier, replacing http://localhost:5000 with the address of the host machine where the Docker container is running. If you are running the container on your local machine and the host's IP address is 127.0.0.1, you can use the following command to make a prediction:

In [ ]:
!curl -X POST -H "Content-Type: application/json" -d '[
  {
    "Pclass": 3,
    "Sex": "male",
    "Age": 22,
    "SibSp": 1,
    "Parch": 0,
    "Fare": 7.25,
    "Embarked": "S"
  },
  {
    "Pclass": 3,
    "Sex": "female",
    "Age": 32,
    "SibSp": 0,
    "Parch": 0,
    "Fare": 7.9250,
    "Embarked": "S"
  }
]' http://127.0.0.1:5000/predict

This should return a JSON object with the predictions for each sample in the input list.

In [ ]:
{
  "prediction": ["0", "1"]
}

# 4.To store the JSON responses to the API request, 

you can use the -o option of curl to specify an output file where the response will be saved. For example, to save the response to a file named response.json, you can use the following command:

In [ ]:
curl -X POST -H "Content-Type: application/json" -d '{
  "Pclass": 3,
  "Sex": "male",
  "Age": 22,
  "SibSp": 1,
  "Parch": 0,
  "Fare": 7.25,
  "Embarked": "S"
}' http://localhost:5000/predict -o response.json

This will send a POST request to the API and save the response to the response.json file. The file will contain the raw JSON response, including the headers and other metadata.

If you want to save only the JSON payload of the response, you can use the jq command-line JSON processor to extract the payload and save it to a file. For example, to save the payload to a file named response_payload.json, you can use the following command:

In [ ]:
curl -X POST -H "Content-Type: application/json" -d '{
  "Pclass": 3,
  "Sex": "male",
  "Age": 22,
  "SibSp": 1,
  "Parch": 0,
  "Fare": 7.25,
  "Embarked": "S"
}' http://localhost:5000/predict | jq '.prediction' > response_payload.json

This will send the same request to the API and save the prediction field of the response to the response_payload.json file.

# 5.My own notes on data type manipulations for handling API input data and feeding into the ML model as dataframe

See more tricks at https://towardsdatascience.com/how-to-convert-json-into-a-pandas-dataframe-100b2ae1e0d8

In [14]:
data = {"data": [5.1, 3.5, 1.4, 0.2]}

input_data = [data["data"]]
input_data

[[5.1, 3.5, 1.4, 0.2]]

In [15]:
type(input_data)

list

In [2]:
import pandas as pd
import numpy

# Method to convert 2D-arrays to JSON via Pandas to serialize autmatically. 
# circumvents error: 'Object of type ndarray is not JSON serializable'

numpyArray = numpy.array([[51, 61, 91], [121, 118, 127]])

df = pd.DataFrame(numpyArray, index=['row 1', 'row 2'], columns=['col 1', 'col 2', 'col 3'])
df_json = df.to_json(orient='index')
df

,col 1,col 2,col 3
row 1,51,61,91
row 2,121,118,127


In [4]:
df_json

'{"row 1":{"col 1":51,"col 2":61,"col 3":91},"row 2":{"col 1":121,"col 2":118,"col 3":127}}'

In [8]:
df2 = pd.DataFrame(numpyArray,  columns=['col 1', 'col 2', 'col 3'])
df_json2 = df2.to_json(orient='index')
df2

,col 1,col 2,col 3
0,51,61,91
1,121,118,127


In [11]:
df_json2

'{"0":{"col 1":51,"col 2":61,"col 3":91},"1":{"col 1":121,"col 2":118,"col 3":127}}'

In [12]:
df_json2[0]

'{'

In [ ]:
df_json2_array_of_dicts = [df_json2.items()]

In [15]:
dataframe = pd.read_json(df_json, orient='index')

In [16]:
dataframe

,col 1,col 2,col 3
row 1,51,61,91
row 2,121,118,127


In [25]:
feature_cols = list(dataframe.columns)
feature_cols

['col 1', 'col 2', 'col 3']

In [19]:
dataframe.to_dict()

{'col 1': {'row 1': 51, 'row 2': 121},
 'col 2': {'row 1': 61, 'row 2': 118},
 'col 3': {'row 1': 91, 'row 2': 127}}

In [21]:
df_dict = dataframe.to_dict("records") # Note this output format is the JSON format (list of dicts with col names as keys) given in the input API
df_dict 

[{'col 1': 51, 'col 2': 61, 'col 3': 91},
 {'col 1': 121, 'col 2': 118, 'col 3': 127}]

To convert to dataframe for conversion we can use

In [23]:
from sklearn.feature_extraction import DictVectorizer
dictvectorizer = DictVectorizer(sort=False)
csr_matrix = dictvectorizer.fit_transform(df_dict)
csr_matrix

<2x3 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [26]:
df_dict_revert = pd.DataFrame(csr_matrix.toarray(),columns=feature_cols)
df_dict_revert

,col 1,col 2,col 3
0,51.0,61.0,91.0
1,121.0,118.0,127.0


Yes, you can deploy a Flask API on Amazon Web Services (AWS). There are several ways to do this, depending on your specific needs and requirements. Here are some options you can consider:

AWS Elastic Beanstalk: This is a fully managed service that makes it easy to deploy and run web applications and services. You can deploy your Flask API to Elastic Beanstalk by packaging it as a Docker container and uploading it to the service. Elastic Beanstalk will then handle the details of deploying and scaling your application.

AWS Lambda: This is a serverless computing platform that allows you to run code in response to events or automatically scale. You can use AWS Lambda to host your Flask API by wrapping it in a function and deploying it to the service. You can then trigger the function using an API Gateway, which provides a fully managed HTTP service for creating, publishing, and maintaining APIs.

AWS EC2: This is a web service that provides resizable compute capacity in the cloud. You can use an EC2 instance to host your Flask API by installing the required dependencies, including Python and Flask, on the instance and running the API script.

I hope this gives you an idea of the options available for deploying a Flask API on AWS. Let me know if you have any questions or need further assistance.





Pls tell me more about how to deploy Flask API in AWS EC2?
To deploy a Flask API on an Amazon EC2 instance, you will need to do the following:

Launch an EC2 instance: First, you will need to launch an EC2 instance using the Amazon EC2 management console, the AWS CLI, or an AWS SDK. Choose an Amazon Machine Image (AMI) that includes the required dependencies, such as Python and Flask, or create your own AMI with the necessary dependencies installed.

Connect to the EC2 instance: Once the instance is launched, you will need to connect to it using Secure Shell (SSH). You will need the instance's public DNS name or IP address, as well as the private key file that you specified when you launched the instance.

Copy the API files to the EC2 instance: Once you are connected to the EC2 instance, you will need to copy the API script and any other necessary files to the instance. You can use a tool such as scp to copy the files over SSH.

Install dependencies: If you are using an AMI that does not include the necessary dependencies, you will need to install them using a package manager such as pip or apt.

Run the API: Finally, you can start the API by running the API script on the EC2 instance. You can use a tool such as nohup or screen to run the script in the background, so that it continues to run even if you close the SSH connection.

That's it! Your Flask API should now be running on the EC2 instance and be accessible from the Internet.

I hope this helps! Let me know if you have any questions or need further assistance.





So the EC2 instance is a VM?
Yes, an Amazon Elastic Compute Cloud (EC2) instance is a virtual machine (VM) provided by the Amazon Web Services (AWS) cloud computing platform. When you launch an EC2 instance, you can choose from a variety of pre-configured templates called Amazon Machine Images (AMIs) that include different operating systems, software packages, and configurations. You can also create your own custom AMI with the specific dependencies that you need.

Once you have launched an EC2 instance, you can connect to it using Secure Shell (SSH) and treat it like a regular server or desktop machine. You can install software, run applications, and store data on the instance, just as you would on a physical machine.

EC2 instances are scalable and flexible, allowing you to choose the right combination of resources (such as CPU, memory, and storage) for your workload. You can also choose the type of instance that best fits your needs, such as a general-purpose instance, a compute-optimized instance, or a GPU-powered instance.

I hope this helps! Let me know if you have any questions or need further assistance.






It's difficult to say which option is the "best" for deploying a Flask API, as it depends on your specific needs and requirements. Here are some factors to consider when choosing which option to use:

AWS Elastic Beanstalk: This is a fully managed service that makes it easy to deploy and run web applications and services. It is a good choice if you want a simple and hassle-free way to deploy your API, as it handles the details of deploying and scaling your application automatically. However, it may not offer as much control or customization as other options.

AWS Lambda: This is a serverless computing platform that allows you to run code in response to events or automatically scale. It is a good choice if you want to pay only for the resources that you consume and do not want to worry about managing servers. However, it may not be suitable for applications that require long-running processes or need to store large amounts of data.

AWS EC2: This is a web service that provides resizable compute capacity in the cloud. It is a good choice if you need full control over the underlying infrastructure and want to customize the operating system, network, and other settings of the instance. However, it requires more maintenance and management than fully managed services such as Elastic Beanstalk.

Certainly! Here are the detailed steps for deploying a Flask API to Amazon Web Services (AWS) Elastic Beanstalk:

Package the API as a Docker container: AWS Elastic Beanstalk supports deploying Docker containers, so you will need to package your Flask API as a Docker container. To do this, you will need to create a Dockerfile that defines the dependencies and steps needed to build the container image.

Build the Docker image: Once you have created the Dockerfile, you can build the Docker image using the docker build command. This will create a local copy of the image that you can use to test and debug the API.

Test the Docker image: Before deploying the Docker image to Elastic Beanstalk, you should test it to make sure that it is working as expected. You can use the docker run command to start a container from the image and test the API locally.

Push the Docker image to Amazon Elastic Container Registry (ECR): AWS Elastic Beanstalk requires that the Docker image be stored in an Amazon Elastic Container Registry (ECR) repository. To push the image to ECR, you will need to create a repository and then use the docker push command to upload the image.

Create an Elastic Beanstalk application: Next, you will need to create an Elastic Beanstalk application using the AWS Management Console or the AWS CLI. This will create the necessary resources and permissions for deploying the API.

Create an Elastic Beanstalk environment: After creating the application, you will need to create an Elastic Beanstalk environment within the application. An environment is a logical deployment group of Amazon EC2 instances, Amazon RDS DB instances, or both. You can choose the type of environment that best fits your needs, such as a web server environment or a worker environment.

Deploy the Docker image to Elastic Beanstalk: Finally, you can deploy the Docker image to Elastic Beanstalk using the AWS Management Console, the AWS CLI, or the Elastic Beanstalk API. Elastic Beanstalk will handle the details of deploying and scaling the API automatically.

That's it! Your Flask API should now be running on Elastic Beanstalk and be accessible from the Internet.

I hope this helps! Let me know if you have any questions or need further assistance.






Here are the detailed steps for deploying a Flask API to Amazon Web Services (AWS) Lambda:

1. Create a Python function: AWS Lambda allows you to run code in response to events or automatically scale. To use Lambda to host your Flask API, you will need to wrap the API in a Python function and define a handler for incoming requests. Here is an example of what the function might look like:

In [ ]:
import json
from flask import Flask, request

app = Flask(__name__)

@app.route("/predict", methods=["POST"])
def predict():
    # Parse the request data
    data = request.get_json()

    # Perform predictions using the model
    predictions = model.predict(data)

    # Return the predictions as a JSON response
    return json.dumps(predictions)

2. Package the function and dependencies: Next, you will need to package the function and any dependencies, such as Flask and the API's model, into a deployable package. You can do this using a tool such as pip to create a requirements.txt file and then using zip to create a zip archive of the files.

3. Create an IAM role: AWS Lambda requires an IAM role that grants the necessary permissions to execute the function. You can create an IAM role using the AWS Management Console or the AWS CLI.

4. Create a Lambda function: After creating the IAM role, you can create a Lambda function using the AWS Management Console, the AWS CLI, or the Lambda API. You will need to provide the deployable package, the IAM role, and the name of the handler function.

5. Create an API Gateway: To make the Lambda function accessible over HTTP, you will need to create an Amazon API Gateway. This will create a fully managed HTTP service for creating, publishing, and maintaining APIs.

6. Create a stage and deployment: After creating the API Gateway, you will need to create a stage and deployment to define how the API should behave. You can specify settings such as the base path, security, and caching options.

7. Test the API: Finally, you can test the API using a tool such as curl or by using the API Gateway's built-in test interface.

That's it! Your Flask API should now be running on Lambda and be accessible through the API Gateway.

Here are the detailed steps for deploying a Flask API to Amazon Web Services (AWS) EC2:

Launch an EC2 instance: First, you will need to launch an EC2 instance using the Amazon EC2 management console, the AWS CLI, or an AWS SDK. Choose an Amazon Machine Image (AMI) that includes the required dependencies, such as Python and Flask, or create your own AMI with the necessary dependencies installed.

Connect to the EC2 instance: Once the instance is launched, you will need to connect to it using Secure Shell (SSH). You will need the instance's public DNS name or IP address, as well as the private key file that you specified when you launched the instance.

Install dependencies: If you are using an AMI that does not include the necessary dependencies, you will need to install them using a package manager such as pip or apt.

Copy the API files to the EC2 instance: Once you are connected to the EC2 instance, you will need to copy the API script and any other necessary files to the instance. You can use a tool such as scp to copy the files over SSH.

Run the API: Finally, you can start the API by running the API script on the EC2 instance. You can use a tool such as nohup or screen to run the script in the background, so that it continues to run even if you close the SSH connection.

That's it! Your Flask API should now be running on the EC2 instance and be accessible from the Internet.